<a href="https://colab.research.google.com/github/agusaaxx/Rekomendasi-NPK-Cabai/blob/main/Rekomendasi_Pupuk.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import joblib
import time
import firebase_admin
from firebase_admin import credentials, db

In [2]:
# ================= KONFIGURASI FIREBASE & PUPUK =================
# Inisialisasi Firebase
cred = credentials.Certificate('firebase-service-key.json') # Pastikan nama file sesuai
firebase_admin.initialize_app(cred, {
    'databaseURL': 'https://<NAMA-PROYEK-ANDA>.firebaseio.com' # GANTI DENGAN URL DATABASE ANDA
})

# Path di Firebase Realtime Database tempat ESP32 mengirim data
FIREBASE_PATH = '/sensor/terbaru' # Sesuaikan dengan path yang Anda gunakan di ESP32

# Konfigurasi pupuk (sama seperti sebelumnya)
KANDUNGAN_N_PUPUK = 16
KANDUNGAN_P_PUPUK = 16
KANDUNGAN_K_PUPUK = 16
BERAT_TANAH_PER_M2 = 260 # kg
# =================================================================

FileNotFoundError: [Errno 2] No such file or directory: 'firebase-service-key.json'

In [11]:
def muat_model():
    """Memuat semua model yang telah dilatih."""
    try:
        model_n = joblib.load('model_rekomendasi_N.pkl')
        model_p = joblib.load('model_rekomendasi_P.pkl')
        model_k = joblib.load('model_rekomendasi_K.pkl')
        print("✅ Semua model machine learning berhasil dimuat.")
        return model_n, model_p, model_k
    except FileNotFoundError:
        print("❌ Error: File model tidak ditemukan. Jalankan script pelatihan terlebih dahulu.")
        return None, None, None


In [12]:
def baca_data_dari_firebase():
    """
    Fungsi baru: Mengambil data sensor terakhir dari Firebase Realtime Database.
    """
    try:
        print(f"\n📡 Mengambil data dari Firebase di path: '{FIREBASE_PATH}'...")
        ref = db.reference(FIREBASE_PATH)
        data_sensor = ref.get()

        if data_sensor:
            print("✔️ Data berhasil diterima dari Firebase.")
            # Pastikan nama field di Firebase cocok dengan yang diharapkan
            # Contoh: { "n_tanah": 50, "p_tanah": 30, ... }
            return data_sensor
        else:
            print("⚠️ Data di Firebase kosong. Menunggu data masuk...")
            return None
    except Exception as e:
        print(f"❌ Gagal terhubung ke Firebase: {e}")
        return None

In [13]:
def hitung_rekomendasi(model_n, model_p, model_k, data_sensor):
    """Menjalankan prediksi dan menghitung dosis pupuk (tidak ada perubahan di fungsi ini)."""
    # 1. Siapkan data input untuk model
    input_prediksi = pd.DataFrame([{
        'kelembaban_tanah': data_sensor['kelembaban_tanah'],
        'suhu_udara': data_sensor['suhu_udara'],
        'kelembaban_udara': data_sensor['kelembaban_udara'],
        'intensitas_cahaya': data_sensor['intensitas_cahaya']
    }])

    # 2. Lakukan prediksi
    n_optimal = model_n.predict(input_prediksi)[0]
    p_optimal = model_p.predict(input_prediksi)[0]
    k_optimal = model_k.predict(input_prediksi)[0]

    print(f"\n--- Hasil Prediksi Model ---")
    print(f"Kadar NPK Optimal Seharusnya: N={n_optimal:.2f} ppm, P={p_optimal:.2f} ppm, K={k_optimal:.2f} ppm")

    # 3. Hitung defisit
    defisit_n = max(0, n_optimal - data_sensor['N_sekarang'])
    defisit_p = max(0, p_optimal - data_sensor['P_sekarang'])
    defisit_k = max(0, k_optimal - data_sensor['K_sekarang'])

    # 4. Hitung kebutuhan pupuk
    kebutuhan_pupuk_untuk_n = (defisit_n * BERAT_TANAH_PER_M2) / (KANDUNGAN_N_PUPUK / 100) / 1000
    kebutuhan_pupuk_untuk_p = (defisit_p * BERAT_TANAH_PER_M2) / (KANDUNGAN_P_PUPUK / 100) / 1000
    kebutuhan_pupuk_untuk_k = (defisit_k * BERAT_TANAH_PER_M2) / (KANDUNGAN_K_PUPUK / 100) / 1000
    dosis_rekomendasi = max(kebutuhan_pupuk_untuk_n, kebutuhan_pupuk_untuk_p, kebutuhan_pupuk_untuk_k)

    print("\n" + "="*40)
    print("      Rekomendasi Pemupukan per m²")
    print("="*40)
    if dosis_rekomendasi > 1: # Beri rekomendasi jika dosis > 1 gram
        print(f"Tambahkan pupuk NPK {KANDUNGAN_N_PUPUK}-{KANDUNGAN_P_PUPUK}-{KANDUNGAN_K_PUPUK} sebanyak: ")
        print(f"                     ✅ {dosis_rekomendasi:.2f} gram ✅")
    else:
        print("✅ Kondisi tanah sudah optimal. Tidak perlu pemupukan.")
    print("="*40)

In [15]:
if __name__ == "__main__":
    model_n, model_p, model_k = muat_model()

    if all([model_n, model_p, model_k]):
        # Loop utama untuk terus memonitor data dari Firebase
        while True:
            data_aktual = baca_data_dari_firebase()
            if data_aktual:
                # Trigger rekomendasi jika data diterima
                hitung_rekomendasi(model_n, model_p, model_k, data_aktual)

            # Beri jeda sebelum pengecekan berikutnya
            print("\nMenunggu pembaruan data berikutnya selama 60 detik...")
            time.sleep(60)

✅ Semua model machine learning berhasil dimuat.
❌ Gagal terhubung ke Firebase: name 'FIREBASE_PATH' is not defined

Menunggu pembaruan data berikutnya selama 60 detik...


KeyboardInterrupt: 